# Load Test on MyAdroit (GPU nodes in the cloud)

In [1]:
%%time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR

CPU times: user 1.61 s, sys: 334 ms, total: 1.94 s
Wall time: 2.15 s


In [2]:
import os
from time import time
start = time()

use_cuda = torch.cuda.is_available()

torch.manual_seed(42)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 64}
test_kwargs  = {'batch_size': 128}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/training_set/", transform=transform)
dataset2 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/test_set/", transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, shuffle=True, **train_kwargs)
test_loader  = torch.utils.data.DataLoader(dataset2, shuffle=True, **test_kwargs)

model = models.resnet50(weights='DEFAULT')
checkpoint = torch.load('/home/jdh4/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth')
model.load_state_dict(checkpoint)
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 128), nn.ReLU(inplace=True), nn.Linear(128, 2))
model = model.to(device)
optimizer = optim.Adadelta(model.fc.parameters(), lr=1.0)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(F.log_softmax(output, dim=1), target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

epochs = 5
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()
    print(f'Train epoch {epoch}')

print(f"{round(time() - start)} seconds", os.environ["USER"], f"cuda={use_cuda}")

Train epoch 1
Train epoch 2
Train epoch 3
Train epoch 4
Train epoch 5
64 seconds jdh4 cuda=True
